In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist

X_train, X_test = X_train/255. , X_test/255.

X_train[0].shape

(28, 28)

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 200)               157000    
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 10)                2010      
                                                                 
Total params: 199210 (778.16 KB)
Trainable params: 199210 (778.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    optimizer = optimizer,
    metrics = ["accuracy"]
)

history = model.fit(X_train, y_train, epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.6689 - accuracy: 0.8315 - val_loss: 0.2886 - val_accuracy: 0.9228
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3040 - accuracy: 0.9137 - val_loss: 0.2246 - val_accuracy: 0.9355
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2516 - accuracy: 0.9268 - val_loss: 0.1912 - val_accuracy: 0.9455


### Power Scheduling

In [4]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, weight_decay=0.0001)
model.compile(
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    optimizer = optimizer,
    metrics = ["accuracy"]
)

history = model.fit(X_train, y_train, epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2173 - accuracy: 0.9375 - val_loss: 0.1670 - val_accuracy: 0.9542
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1920 - accuracy: 0.9447 - val_loss: 0.1509 - val_accuracy: 0.9587
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1712 - accuracy: 0.9507 - val_loss: 0.1388 - val_accuracy: 0.9612


### Exponential Scheduling

In [5]:
def exponential_decay_fn(epoch, lr):
    return lr * 0.1 ** (epoch / 20)

initial_learning_rate = 0.01
optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: exponential_decay_fn(epoch, initial_learning_rate)
)

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=optimizer,
    metrics=["accuracy"]
)

history = model.fit(X_train, y_train, epochs=3, validation_split=0.1, callbacks=[lr_scheduler])

Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1541 - accuracy: 0.9557 - val_loss: 0.1281 - val_accuracy: 0.9668 - lr: 0.0100
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1394 - accuracy: 0.9602 - val_loss: 0.1200 - val_accuracy: 0.9685 - lr: 0.0089
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1284 - accuracy: 0.9637 - val_loss: 0.1133 - val_accuracy: 0.9685 - lr: 0.0079


In [7]:
# learing rate schedules
history.history["lr"]

[0.01, 0.008912509, 0.007943282]

### Piecewise Constant Scheduling

In [15]:
def piecewise_constant_lr_fn(epoch):
    if epoch<3:
        return 0.01
    elif epoch<15:
        return 0.005
    else:
        return 0.001
    
    
optimizer = tf.keras.optimizers.SGD()

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: piecewise_constant_lr_fn(epoch)
)

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=optimizer,
    metrics=["accuracy"]
)

history = model.fit(X_train, y_train, epochs=5, validation_split=0.1, callbacks=[lr_scheduler])

Epoch 1/5
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0528 - accuracy: 0.9858 - val_loss: 0.0762 - val_accuracy: 0.9792 - lr: 0.0100
Epoch 2/5
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0500 - accuracy: 0.9865 - val_loss: 0.0788 - val_accuracy: 0.9787 - lr: 0.0100
Epoch 3/5
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0472 - accuracy: 0.9876 - val_loss: 0.0788 - val_accuracy: 0.9785 - lr: 0.0100
Epoch 4/5
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0431 - accuracy: 0.9891 - val_loss: 0.0741 - val_accuracy: 0.9800 - lr: 0.0050
Epoch 5/5
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0415 - accuracy: 0.9897 - val_loss: 0.0729 - val_accuracy: 0.9808 - lr: 0.0050


In [16]:
history.history["lr"]

[0.01, 0.01, 0.01, 0.005, 0.005]

### Performance Scheduling

In [17]:
import math

batch_size = 32
n_epochs = 15
n_steps = n_epochs * math.ceil(len(X_train)/batch_size)

schedule_learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01, decay_steps=n_steps, decay_rate=0.1
)

optimizer = tf.keras.optimizers.SGD(learning_rate=schedule_learning_rate)

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=optimizer,
    metrics=["accuracy"]
)

history = model.fit(X_train, y_train, epochs=15, validation_split=0.1)

Epoch 1/15
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0421 - accuracy: 0.9894 - val_loss: 0.0750 - val_accuracy: 0.9795
Epoch 2/15
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0397 - accuracy: 0.9902 - val_loss: 0.0726 - val_accuracy: 0.9807
Epoch 3/15
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0374 - accuracy: 0.9913 - val_loss: 0.0730 - val_accuracy: 0.9803
Epoch 4/15
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0358 - accuracy: 0.9915 - val_loss: 0.0720 - val_accuracy: 0.9795
Epoch 5/15
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0342 - accuracy: 0.9922 - val_loss: 0.0719 - val_accuracy: 0.9800
Epoch 6/15
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0331 - accuracy: 0.9926 - val_loss: 0.0713 - val_accuracy: 0.9805
Epoch 7/15
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0322 - accuracy: 0.9932 - val_loss: 0.0721 - val_accuracy:

Performance Scheduling is consideded one of the best learning rate schedulin.